@author  Akanksha Joshi

In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Input data files are available in the "../input/" directory.

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

188-million-us-wildfires



### Establishing a connection with the database file.

In [3]:
cnx = sqlite3.connect('/kaggle/input/188-million-us-wildfires/FPA_FOD_20170508.sqlite')

In [4]:
fires = pd.read_sql("""SELECT * FROM fires""", con=cnx)

In [5]:
fires.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...


### Dealing with Duplicate Data

In [6]:
fires.drop_duplicates() # removing the duplicate rows

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,1880461,300348363,2015CAIRS29019636,NONFED,ST-CACDF,ST/C&L,USCASHU,Shasta-Trinity Unit,CASHU,Shasta-Trinity Unit,...,A,40.481637,-122.389375,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00P\xb8\x1e\x85\xeb\x9...
1880461,1880462,300348373,2015CAIRS29217935,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...
1880462,1880463,300348375,2015CAIRS28364460,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,A,37.617619,-120.938570,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00\x00\x80\xbe\x88\x11...
1880463,1880464,300348377,2015CAIRS29218079,NONFED,ST-CACDF,ST/C&L,USCATCU,Tuolumne-Calaveras Unit,CATCU,Tuolumne-Calaveras Unit,...,B,37.672235,-120.898356,12.0,MUNICIPAL/LOCAL,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00x\xba_\xaa~9^\xc0\xb...


## Dealing with Missing Data

In [7]:
fires.isna().sum()

OBJECTID                            0
FOD_ID                              0
FPA_ID                              0
SOURCE_SYSTEM_TYPE                  0
SOURCE_SYSTEM                       0
NWCG_REPORTING_AGENCY               0
NWCG_REPORTING_UNIT_ID              0
NWCG_REPORTING_UNIT_NAME            0
SOURCE_REPORTING_UNIT               0
SOURCE_REPORTING_UNIT_NAME          0
LOCAL_FIRE_REPORT_ID          1459286
LOCAL_INCIDENT_ID              820821
FIRE_CODE                     1555636
FIRE_NAME                      957189
ICS_209_INCIDENT_NUMBER       1854748
ICS_209_NAME                  1854748
MTBS_ID                       1869462
MTBS_FIRE_NAME                1869462
COMPLEX_NAME                  1875282
FIRE_YEAR                           0
DISCOVERY_DATE                      0
DISCOVERY_DOY                       0
DISCOVERY_TIME                 882638
STAT_CAUSE_CODE                     0
STAT_CAUSE_DESCR                    0
CONT_DATE                      891531
CONT_DOY    

In [8]:
## removing columns which won't contribute to our analysis (such as ID's and code's) 

In [9]:
fires = fires.drop(['LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID','ICS_209_INCIDENT_NUMBER','ICS_209_NAME','MTBS_ID','MTBS_FIRE_NAME','COMPLEX_NAME','FIRE_CODE'], axis=1)

In [10]:
fires.shape

(1880465, 31)

In [11]:
fires.isna().sum()

OBJECTID                           0
FOD_ID                             0
FPA_ID                             0
SOURCE_SYSTEM_TYPE                 0
SOURCE_SYSTEM                      0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNIT_ID             0
NWCG_REPORTING_UNIT_NAME           0
SOURCE_REPORTING_UNIT              0
SOURCE_REPORTING_UNIT_NAME         0
FIRE_NAME                     957189
FIRE_YEAR                          0
DISCOVERY_DATE                     0
DISCOVERY_DOY                      0
DISCOVERY_TIME                882638
STAT_CAUSE_CODE                    0
STAT_CAUSE_DESCR                   0
CONT_DATE                     891531
CONT_DOY                      891531
CONT_TIME                     972173
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_CODE                         0
OWNER_DESCR                        0
STATE                              0
C

In [12]:
fires['FIRE_NAME'].sample(25)

1859666                 RIGHT OF WAY
1231355                         None
1067480    UNION RD / AIRLINE HWY   
1237582             FY2000-FLOYD-102
729759                          None
1381723                         None
743859                          None
830285                          None
648524                          None
365081                    MASHPEE WF
712442                 FAIRVIEW ROAD
268834                     040000038
1501610                         None
480419                          None
1590921                         None
1253206            FY2002-COWETA-115
1429466                         None
432632                          None
1717551                         None
967895                          None
1674841             SER-20131023-001
1241818              FY2000-WARE-168
898251                          None
297254                        LIQUOR
816546                          None
Name: FIRE_NAME, dtype: object

In [13]:
fires['FIRE_NAME'].unique()

array(['FOUNTAIN', 'PIGEON', 'SLACK', ..., '1-64', 'ODESSA 2',
       'BARKER BL  BIG_BEAR_LAKE_'], dtype=object)

In [14]:
fires.shape

(1880465, 31)

In [15]:
fires['FIRE_NAME'] = fires['FIRE_NAME'].fillna('Unknown')

In [16]:
fires['FIRE_NAME'].sample(25)

989382                       Unknown
960848                       Unknown
1289613                      Unknown
165818                   DANCING DOG
712                             0282
955096                       Unknown
89029                     SCOTT ROAD
993382                       Unknown
1263430                      Unknown
1411995                      Unknown
1707226                      Unknown
830927                       Unknown
1256037           FY2002-LAURENS-189
308338                          SAME
184668                          R459
1850512                         OSLO
1142493    LOCAL                    
415787                    BROKEN BOW
1328241                      Unknown
1072658    CLOVIS                   
799922                       Unknown
282653                     COMMUINTY
1634575                   WOODS FIRE
1823539                      Unknown
1418860                      Unknown
Name: FIRE_NAME, dtype: object

In [17]:
fires.isna().sum()

OBJECTID                           0
FOD_ID                             0
FPA_ID                             0
SOURCE_SYSTEM_TYPE                 0
SOURCE_SYSTEM                      0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNIT_ID             0
NWCG_REPORTING_UNIT_NAME           0
SOURCE_REPORTING_UNIT              0
SOURCE_REPORTING_UNIT_NAME         0
FIRE_NAME                          0
FIRE_YEAR                          0
DISCOVERY_DATE                     0
DISCOVERY_DOY                      0
DISCOVERY_TIME                882638
STAT_CAUSE_CODE                    0
STAT_CAUSE_DESCR                   0
CONT_DATE                     891531
CONT_DOY                      891531
CONT_TIME                     972173
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_CODE                         0
OWNER_DESCR                        0
STATE                              0
C

In [18]:
bool_series = pd.isnull(fires[['COUNTY','FIPS_CODE','FIPS_NAME']])
bool_series.sample(50)

,COUNTY,FIPS_CODE,FIPS_NAME
416331,False,False,False
1615551,False,False,False
524768,False,False,False
190640,True,True,True
346825,False,False,False
66834,True,True,True
1819868,False,False,False
1798563,True,True,True
61804,True,True,True
1402369,False,False,False


### we can drop county and fips code because we get the same info from fips name

In [19]:
fires = fires.drop(['COUNTY','FIPS_CODE'],axis=1)

In [20]:
fires.isna().sum()

OBJECTID                           0
FOD_ID                             0
FPA_ID                             0
SOURCE_SYSTEM_TYPE                 0
SOURCE_SYSTEM                      0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNIT_ID             0
NWCG_REPORTING_UNIT_NAME           0
SOURCE_REPORTING_UNIT              0
SOURCE_REPORTING_UNIT_NAME         0
FIRE_NAME                          0
FIRE_YEAR                          0
DISCOVERY_DATE                     0
DISCOVERY_DOY                      0
DISCOVERY_TIME                882638
STAT_CAUSE_CODE                    0
STAT_CAUSE_DESCR                   0
CONT_DATE                     891531
CONT_DOY                      891531
CONT_TIME                     972173
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_CODE                         0
OWNER_DESCR                        0
STATE                              0
F

In [21]:
fires['FIPS_NAME'] = fires['FIPS_NAME'].fillna('Unknown')

In [22]:
fires.isna().sum()

OBJECTID                           0
FOD_ID                             0
FPA_ID                             0
SOURCE_SYSTEM_TYPE                 0
SOURCE_SYSTEM                      0
NWCG_REPORTING_AGENCY              0
NWCG_REPORTING_UNIT_ID             0
NWCG_REPORTING_UNIT_NAME           0
SOURCE_REPORTING_UNIT              0
SOURCE_REPORTING_UNIT_NAME         0
FIRE_NAME                          0
FIRE_YEAR                          0
DISCOVERY_DATE                     0
DISCOVERY_DOY                      0
DISCOVERY_TIME                882638
STAT_CAUSE_CODE                    0
STAT_CAUSE_DESCR                   0
CONT_DATE                     891531
CONT_DOY                      891531
CONT_TIME                     972173
FIRE_SIZE                          0
FIRE_SIZE_CLASS                    0
LATITUDE                           0
LONGITUDE                          0
OWNER_CODE                         0
OWNER_DESCR                        0
STATE                              0
F

In [23]:
fires = fires.dropna(subset=['CONT_DATE', 'CONT_DOY'])

In [24]:
fires.isna().sum()

OBJECTID                          0
FOD_ID                            0
FPA_ID                            0
SOURCE_SYSTEM_TYPE                0
SOURCE_SYSTEM                     0
NWCG_REPORTING_AGENCY             0
NWCG_REPORTING_UNIT_ID            0
NWCG_REPORTING_UNIT_NAME          0
SOURCE_REPORTING_UNIT             0
SOURCE_REPORTING_UNIT_NAME        0
FIRE_NAME                         0
FIRE_YEAR                         0
DISCOVERY_DATE                    0
DISCOVERY_DOY                     0
DISCOVERY_TIME                77329
STAT_CAUSE_CODE                   0
STAT_CAUSE_DESCR                  0
CONT_DATE                         0
CONT_DOY                          0
CONT_TIME                     81022
FIRE_SIZE                         0
FIRE_SIZE_CLASS                   0
LATITUDE                          0
LONGITUDE                         0
OWNER_CODE                        0
OWNER_DESCR                       0
STATE                             0
FIPS_NAME                   

In [25]:
fires = fires.dropna(subset=['CONT_TIME','DISCOVERY_TIME'])

In [26]:
fires.isna().sum()

OBJECTID                      0
FOD_ID                        0
FPA_ID                        0
SOURCE_SYSTEM_TYPE            0
SOURCE_SYSTEM                 0
NWCG_REPORTING_AGENCY         0
NWCG_REPORTING_UNIT_ID        0
NWCG_REPORTING_UNIT_NAME      0
SOURCE_REPORTING_UNIT         0
SOURCE_REPORTING_UNIT_NAME    0
FIRE_NAME                     0
FIRE_YEAR                     0
DISCOVERY_DATE                0
DISCOVERY_DOY                 0
DISCOVERY_TIME                0
STAT_CAUSE_CODE               0
STAT_CAUSE_DESCR              0
CONT_DATE                     0
CONT_DOY                      0
CONT_TIME                     0
FIRE_SIZE                     0
FIRE_SIZE_CLASS               0
LATITUDE                      0
LONGITUDE                     0
OWNER_CODE                    0
OWNER_DESCR                   0
STATE                         0
FIPS_NAME                     0
Shape                         0
dtype: int64

## Encoding categorical data

In [27]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892007 entries, 0 to 1880460
Data columns (total 29 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   OBJECTID                    892007 non-null  int64  
 1   FOD_ID                      892007 non-null  int64  
 2   FPA_ID                      892007 non-null  object 
 3   SOURCE_SYSTEM_TYPE          892007 non-null  object 
 4   SOURCE_SYSTEM               892007 non-null  object 
 5   NWCG_REPORTING_AGENCY       892007 non-null  object 
 6   NWCG_REPORTING_UNIT_ID      892007 non-null  object 
 7   NWCG_REPORTING_UNIT_NAME    892007 non-null  object 
 8   SOURCE_REPORTING_UNIT       892007 non-null  object 
 9   SOURCE_REPORTING_UNIT_NAME  892007 non-null  object 
 10  FIRE_NAME                   892007 non-null  object 
 11  FIRE_YEAR                   892007 non-null  int64  
 12  DISCOVERY_DATE              892007 non-null  float64
 13  DISCOVERY_DOY

In [28]:
fips_le = LabelEncoder()
fips_labels = fips_le.fit_transform(fires.FIRE_SIZE_CLASS)
fires["FIRE_SIZE_CLASS_ENCODED"] = fips_labels

In [29]:
encoded_cols = pd.get_dummies(fires[['STAT_CAUSE_DESCR', 'FIPS_NAME']], columns=['STAT_CAUSE_DESCR', 'FIPS_NAME'])
fires = pd.concat([fires, encoded_cols], axis=1)

In [30]:
fires.sample(20)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIPS_NAME_Yellowstone,FIPS_NAME_Yoakum,FIPS_NAME_Yolo,FIPS_NAME_York,FIPS_NAME_Young,FIPS_NAME_Yuba,FIPS_NAME_Yukon-Koyukuk,FIPS_NAME_Yuma,FIPS_NAME_Zapata,FIPS_NAME_Zavala
13624,13625,13685,FS-1442296,FED,FS-FIRESTAT,FS,USIDSTF,Sawtooth National Forest,0414,Sawtooth National Forest,...,0,0,0,0,0,0,0,0,0,0
1176645,1176646,1427784,FS-1484906,FED,FS-FIRESTAT,FS,USMIHMF,Huron-Manistee National Forest,0904,Huron-Manistee National Forest,...,0,0,0,0,0,0,0,0,0,0
162430,162431,164018,FS-392046,FED,FS-FIRESTAT,FS,USNVHTF,Humboldt-Toiyabe National Forest,0417,Humboldt-Toiyabe National Forest,...,0,0,0,0,0,0,0,0,0,0
180174,180175,182318,W-312400,FED,DOI-WFMI,BIA,USSDPRA,Pine Ridge Agency,SDPRA,Pine Ridge Agency,...,0,0,0,0,0,0,0,0,0,0
5056,5057,5080,FS-1428723,FED,FS-FIRESTAT,FS,USKYDBF,Daniel Boone National Forest,0802,Daniel Boone National Forest,...,0,0,0,0,0,0,0,0,0,0
1796928,1796929,300208674,W-678269,FED,DOI-WFMI,BIA,USAZHOA,Hopi Agency,AZHOA,Hopi Agency,...,0,0,0,0,0,0,0,0,0,0
4765,4766,4788,FS-1428183,FED,FS-FIRESTAT,FS,USAZKNF,Kaibab National Forest,0307,Kaibab National Forest,...,0,0,0,0,0,0,0,0,0,0
251148,251149,256070,W-582860,FED,DOI-WFMI,BIA,USOKTLA,Talihina Agency,OKTLA,Talihina Agency,...,0,0,0,0,0,0,0,0,0,0
125414,125415,126648,FS-351759,FED,FS-FIRESTAT,FS,USSCFMF,Francis Marion & Sumter National Forests,0812,Francis Marion-Sumter National Forest,...,0,0,0,0,0,0,0,0,0,0
339727,339728,348268,W-558015,FED,DOI-WFMI,BLM,USNMROD,Roswell Field Office,NMROD,Roswell Field Office,...,0,0,0,0,0,0,0,0,0,0


In [31]:
fires.shape

(892007, 1584)

In [32]:
# fires.sample(10)

In [33]:
fires.columns

Index(['OBJECTID', 'FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM',
       'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID',
       'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT',
       'SOURCE_REPORTING_UNIT_NAME',
       ...
       'FIPS_NAME_Yellowstone', 'FIPS_NAME_Yoakum', 'FIPS_NAME_Yolo',
       'FIPS_NAME_York', 'FIPS_NAME_Young', 'FIPS_NAME_Yuba',
       'FIPS_NAME_Yukon-Koyukuk', 'FIPS_NAME_Yuma', 'FIPS_NAME_Zapata',
       'FIPS_NAME_Zavala'],
      dtype='object', length=1584)

In [34]:
fires['STAT_CAUSE_DESCR'].unique()

array(['Miscellaneous', 'Lightning', 'Debris Burning', 'Campfire',
       'Equipment Use', 'Arson', 'Children', 'Railroad', 'Smoking',
       'Powerline', 'Structure', 'Fireworks', 'Missing/Undefined'],
      dtype=object)

In [35]:
fires['FIRE_SIZE_CLASS_ENCODED'].unique()

array([0, 1, 6, 2, 3, 5, 4])

## Splitting dataframe into Train-Test dataframes

In [36]:
# Train - test split on entire dataframe

train_df,test_df = train_test_split(fires, test_size=0.2, random_state=42)

In [37]:
train_df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIPS_NAME_Yellowstone,FIPS_NAME_Yoakum,FIPS_NAME_Yolo,FIPS_NAME_York,FIPS_NAME_Young,FIPS_NAME_Yuba,FIPS_NAME_Yukon-Koyukuk,FIPS_NAME_Yuma,FIPS_NAME_Zapata,FIPS_NAME_Zavala
1623286,1623287,201658541,SFO-NY-2012-NY4414-2012-050012,NONFED,ST-NASF,ST/C&L,USNYNYX,Fire Department of New York,NY4414,Fire Department of New York,...,0,0,0,0,0,0,0,0,0,0
1569450,1569451,201435787,FS-1509063,FED,FS-FIRESTAT,FS,USORUPF,Umpqua National Forest,0615,Umpqua National Forest,...,0,0,0,0,0,0,0,0,0,0
329242,329243,336663,W-522697,FED,DOI-WFMI,BLM,USIDTFD,Twin Falls District,IDJRD,Jarbidge Field Office,...,0,0,0,0,0,0,0,0,0,0
1218663,1218664,1476284,W-603889,FED,DOI-WFMI,BLM,USUTSLD,Salt Lake Field Office,UTSLD,Salt Lake Field Office,...,0,0,0,0,0,0,0,0,0,0
1749208,1749209,300090002,SFO-2014GAGAS-FY2014-Wilcox-004,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261366,261367,266414,W-536509,FED,DOI-WFMI,BIA,USSDCRA,Cheyenne River Agency,SDCRA,Cheyenne River Agency,...,0,0,0,0,0,0,0,0,0,0
372212,372213,384326,FWS-2007MSMSRC9G9,FED,FWS-FMIS,FWS,USMSMSR,Mississippi Sandhill Crane National Wildlife R...,MSMSR,Mississippi Sandhill Crane National Wildlife R...,...,0,0,0,0,0,0,0,0,0,0
132151,132152,133461,FS-358879,FED,FS-FIRESTAT,FS,USIDSCF,Salmon-Challis National Forest,0413,Salmon-Challis National Forest,...,0,0,0,0,0,0,0,0,0,0
1498455,1498456,201171855,2011SCSCS12FF0002,NONFED,ST-NASF,ST/C&L,USSCSCS,South Carolina Forestry Commission,SCSCS,South Carolina Forestry Commission,...,0,0,0,0,0,0,0,0,0,0


In [38]:
test_df

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIPS_NAME_Yellowstone,FIPS_NAME_Yoakum,FIPS_NAME_Yolo,FIPS_NAME_York,FIPS_NAME_Young,FIPS_NAME_Yuba,FIPS_NAME_Yukon-Koyukuk,FIPS_NAME_Yuma,FIPS_NAME_Zapata,FIPS_NAME_Zavala
136716,136717,138068,FS-364062,FED,FS-FIRESTAT,FS,USORDEF,Deschutes National Forest,0601,Deschutes National Forest,...,0,0,0,0,0,0,0,0,0,0
135095,135096,136434,FS-362140,FED,FS-FIRESTAT,FS,USMSMNF,National Forests in Mississippi,0807,National Forests in Mississippi,...,0,0,0,0,0,0,0,0,0,0
1810147,1810148,300225511,SFO-2015KS00002780,NONFED,ST-NASF,ST/C&L,USKSKSS,Kansas State Forestry,KSKSS,Kansas State Forestry (PBC Area),...,0,0,0,0,0,0,0,0,0,0
150448,150449,151906,FS-378935,FED,FS-FIRESTAT,FS,USORWWF,Wallowa-Whitman National Forest,0616,Wallowa-Whitman National Forest,...,0,0,0,0,0,0,0,0,0,0
239191,239192,243568,W-257052,FED,DOI-WFMI,NPS,USCOCCP,Curecanti National Recreation Area,COCCP,Curecanti National Recreation Area,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228519,1228520,1618160,SFO-OK-1-2010-10015-A,NONFED,ST-NASF,ST/C&L,USOKOKS,Oklahoma Division of Forestry,OKOFS,Oklahoma Forestry Services,...,0,0,0,0,0,0,0,0,0,0
1387989,1387990,19083789,SFO-GA-MOR-11-2/16/1996-1830,NONFED,ST-GAGAS,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,0,0,0,0,0,0,0,0,0,0
111634,111635,112796,FS-337809,FED,FS-FIRESTAT,FS,USCASNF,Sierra National Forest,0515,Sierra National Forest,...,0,0,0,0,0,0,0,0,0,0
125522,125523,126756,FS-351872,FED,FS-FIRESTAT,FS,USNMSNF,Santa Fe National Forest,0310,Santa Fe National Forest,...,0,0,0,0,0,0,0,0,0,0


### Saving dataframe as CSV

In [39]:
train_df.to_csv('train.csv',index=False)

In [40]:
test_df.to_csv('test.csv',index=False)

### Saving csv without encodings (to use for ease during EDA)

In [41]:
cols = [c for c in train_df.columns if not c.startswith('FIPS_NAME_') and not c.startswith('STAT_CAUSE_DESCR_')]
cols
train_df_no_encode = train_df[cols]

In [42]:
train_df_no_encode.head()

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,FIPS_NAME,Shape,FIRE_SIZE_CLASS_ENCODED
1623286,1623287,201658541,SFO-NY-2012-NY4414-2012-050012,NONFED,ST-NASF,ST/C&L,USNYNYX,Fire Department of New York,NY4414,Fire Department of New York,...,0.1,A,41.147000,-73.927000,14.0,MISSING/NOT SPECIFIED,NY,Rockland,b'\x00\x01\xad\x10\x00\x00\x14\xd9\xce\xf7S{R\...,0
1569450,1569451,201435787,FS-1509063,FED,FS-FIRESTAT,FS,USORUPF,Umpqua National Forest,0615,Umpqua National Forest,...,0.1,A,43.084444,-122.617778,5.0,USFS,OR,Douglas,b'\x00\x01\xad\x10\x00\x00\xe8R\xd0\xab\x89\xa...,0
329242,329243,336663,W-522697,FED,DOI-WFMI,BLM,USIDTFD,Twin Falls District,IDJRD,Jarbidge Field Office,...,98.6,C,42.741900,-114.848900,1.0,BLM,ID,Twin Falls,b'\x00\x01\xad\x10\x00\x00\xc0d\xaa`T\xb6\\\xc...,2
1218663,1218664,1476284,W-603889,FED,DOI-WFMI,BLM,USUTSLD,Salt Lake Field Office,UTSLD,Salt Lake Field Office,...,0.5,B,40.293300,-111.977500,8.0,PRIVATE,UT,Utah,b'\x00\x01\xad\x10\x00\x00\xc0\xf5(\\\x8f\xfe[...,1
1749208,1749209,300090002,SFO-2014GAGAS-FY2014-Wilcox-004,NONFED,ST-NASF,ST/C&L,USGAGAS,Georgia Forestry Commission,GAGAS,Georgia Forestry Commission,...,11.8,C,31.949458,-83.289372,8.0,PRIVATE,GA,Wilcox,b'\x00\x01\xad\x10\x00\x00TQ\x0f\x13\x85\xd2T\...,2


In [43]:
train_df_no_encode.to_csv('train_no_encoding.csv',index=False)

In [44]:
cols = [c for c in test_df.columns if not c.startswith('FIPS_NAME_') and not c.startswith('STAT_CAUSE_DESCR_')]
cols
test_df_no_encode = test_df[cols]

In [45]:
test_df_no_encode.to_csv('test_no_encoding.csv',index=False)